##Processing for organization dataset
Authors: Sanjana and Rachel

Needed files: 

*   interest_areas.txt
*   interest_categories.txt
*   org_dataset.csv
*   encoded_org_dataset.csv




####Import libraries

In [ ]:
import pandas as pd
import numpy as np
from numpy.random import choice
import copy
import re
import io

### Process age columns into single majority age group designation
Author: Sanjana

In [ ]:
df = pd.read_csv('/content/org_dataset.csv')

FileNotFoundError: ignored

In [ ]:
print(f"df type: {type(df)}")
print(f"df shape: {df.shape}")

In [ ]:
df.columns

Deep copy selected columns 

In [ ]:
organization = copy.deepcopy(df['Organization/ City Agency/ Division Name'])
total_vol = copy.deepcopy(df['Total Vounteers'])
youth_vol = copy.deepcopy(df['Youth volunteers'])
adult_vol = copy.deepcopy(df['Adult Volunteers'])
older_vol = copy.deepcopy(df['Older adult Volunteers'])

Clean up volunteer age data 

In [ ]:
for i in range(len(total_vol)):
  total_vol[i] = total_vol[i].replace(',', '')
  youth_vol[i] = youth_vol[i].replace(',','')
  youth_vol[i] = youth_vol[i].replace('-','0')
  adult_vol[i] = adult_vol[i].replace(',','')
  adult_vol[i] = adult_vol[i].replace('-','0')
  older_vol[i] = older_vol[i].replace(',','')
  older_vol[i] = older_vol[i].replace('-','0')

Lists

In [ ]:
total_vol = [float(i) for i in total_vol]
youth_vol = [float(i) for i in youth_vol]
adult_vol = [float(i) for i in adult_vol]
older_vol = [float(i) for i in older_vol]

organizationZero =[]

Delete all zero entries from lists

Keep track of totall zero entries

In [ ]:
total_Zeros = 0

for i in range(len(total_vol)-1, -1, -1):     # iterate backwards
  if youth_vol[i] == 0 and adult_vol[i] == 0 and older_vol[i] == 0: 

    total_Zeros +=1
    organizationZero.append(organization[i])
    

    del(total_vol[i])
    del(youth_vol[i])
    del(adult_vol[i])
    del(older_vol[i])
    del(organization[i])

Correct given age data

In [ ]:
dictIn = {"Total volunteers" : total_vol, "Youth volunteers": youth_vol, "Adult volunteers" : adult_vol, "Older volunteers" : older_vol}

In [ ]:
df2 = pd.DataFrame(dictIn)
df2_vols = df2.loc[:,df2.columns != 'Total volunteers']

Get max volunteers in age groups for each organization 

In [ ]:
maxList = []
maxList = re.findall('[A-Z][^A-Z]*',df2_vols.idxmax(axis=1).sum())

In [ ]:
for i in maxList:
  if (i == 'Youth volunteers'):
    maxList[maxList.index(i)] = 0
  
  elif(i == 'Adult volunteers'):
   maxList[maxList.index(i)] = 1

  elif(i == 'Older volunteers'):
    maxList[maxList.index(i)] = 2


Generate Age data for Zero entries with respect to age demographics in NYC

In [ ]:
from numpy.random import choice

ageList = [0,1,2]

generatedAgeArray = choice(ageList,total_Zeros, p=[0.10,0.78,0.12]) 

Join given and generated data  

In [ ]:
for i in generatedAgeArray:
  maxList.append(generatedAgeArray[i])


In [ ]:
organizationList = []
for i in organization:
  organizationList.append(str(i))

for i in organizationZero:
  organizationList.append(i)

In [ ]:
dictAge = {"Organization/ City Agency/ Division Name" : organizationList, "Most volunteers in age group" : maxList}

In [ ]:
df_finalAge = pd.DataFrame(dictAge)

Final age dataset 

In [ ]:
df_finalAge

### Process interests into categories
Author: Rachel

In [ ]:
# open files
areas_file = open('/content/interest_areas.txt', 'r')   # list of all interests w/ category designations
cat_file = open('/content/interest_categories.txt', 'r')    # list of all categories w/ numbers

####Process interest areas list

In [ ]:
# for areas file
lines = areas_file.read()
interests_list = lines.split('\n')

In [ ]:
for i in range(len(interests_list)):
  interests_list[i] = interests_list[i].split('=')
  interests_list[i][1] = interests_list[i][1].split(',')

interests_list

In [ ]:
# create list with all interests for each category
categories_list = []

for i in range(10):
  temp_list = []
  for j in range(len(interests_list)):
    if str(i+1) in interests_list[j][1]:
      temp_list.append(interests_list[j][0])
  categories_list.append(temp_list)

categories_list

####Process category file

In [ ]:
lines = cat_file.read()
short_cat_list = lines.split('\n')

for i in range(len(short_cat_list)):
  short_cat_list[i] = short_cat_list[i].split('*')

short_cat_list

####Modify dataset

In [ ]:
org_df = pd.read_csv('/content/org_dataset.csv')

In [ ]:
interest_areas = org_df['Interest Areas']
cat_col = []

for i in range(len(interest_areas)):
  temp_list = []
  interests = ''
  for entry in categories_list:   # go through each entry in the interests listed by category
    for item in entry:    # go through every interest area in each category
      if item in repr(interest_areas[i]):
        temp_list.append(short_cat_list[categories_list.index(entry)][1])

  # convert to dictionary and back to remove duplicates     
  temp_dict = dict.fromkeys(temp_list)
  temp_list = temp_dict.keys()
  
  # put into string separated by commas
  for interest in temp_list:
    interests = interests + interest + ';'
  cat_col.append(interests.rstrip(';'))

# list type
cat_col

In [ ]:
org_df['Interest Categories'] = cat_col
org_df.head()

In [ ]:
areas_file.close()
cat_file.close()

### Combining dataframes for final organization dataset
Authors: Sanjana and Rachel

#### Merge dataframes

In [ ]:
df4 = pd.merge(org_df,df_finalAge, on="Organization/ City Agency/ Division Name")

df4.head()

In [ ]:
df4.columns

In [ ]:
df4.drop(['Street Address/Mailing Address', 'City', 'State', 'Postcode',
       'Year Surveyed', 'Total Vounteers', 'Youth volunteers',
       'Adult Volunteers', 'Older adult Volunteers', 'Interest Areas', 'Special Populations Served', 'Boroughs  Served',
       'Latitude', 'Longitude', 'Community Board',
       'Council District', 'Census Tract', 'BIN', 'BBL', 'NTA'], axis=1, inplace=True)

Note: column 'Organization Type" could be useful for displaying in front-end

In [ ]:
final_df = df4.loc[:,df4.columns != 'Organization Type']

####Handling missing values

In [ ]:
final_df.isna().sum()

In [ ]:
borough = copy.deepcopy(final_df['Borough'])

Boroughs of missing values with respect to locations of organizations 

In [ ]:
generatedBoroughList = [ "MANHATTAN", "BROOKLYN", "QUEENS", "MANHATTAN", "BRONX", "BROOKLYN", "BROOKLYN", "BRONX", "QUEENS", "BROOKLYN", "QUEENS", "MANHATTAN","MANHATTAN","MANHATTAN","MANHATTAN","MANHATTAN","MANHATTAN","BRONX", "QUEENS", "QUEENS", "BROOKLYN", "MANHATTAN","BRONX","MANHATTAN","MANHATTAN","MANHATTAN","MANHATTAN","MANHATTAN","BRONX", "QUEENS", "MANHATTAN", "BROOKLYN", "MANHATTAN","MANHATTAN", "QUEENS", "QUEENS","QUEENS","QUEENS","BRONX","MANHATTAN","MANHATTAN", "MANHATTAN","QUEENS", "MANHATTAN"]
                        

In [ ]:
naNIndex = []
for i in range(len(borough)):
  if not (isinstance(borough[i],str)):
    naNIndex.append(i)


Insert researched boroughs into borough list

In [ ]:
count = 0
for i in naNIndex:
  if count < 44:
    borough[i] = generatedBoroughList[count]
    count +=1

In [ ]:
boroughList = []
for i in range(len(borough)):
    boroughList.append(borough[i])

Remove previous Borough column with missing values

In [ ]:
final_df.drop('Borough', axis=1, inplace=True)

Insert new Borough column 

In [ ]:
final_df.insert(1,'Borough',boroughList)

Check

In [ ]:
final_df.isna().sum()

Final pre encoded organization dataset

In [ ]:
final_df

#### Save to CSV file

In [ ]:
#final_df.to_csv("/content/Preencoding_final_dataframe.csv")

In [ ]:
#final_df = pd.read_csv("/content/Preencoding_final_dataframe.csv")

####One-hot Encoding

Encoding boroughs

In [ ]:
one_hotBoroughs = pd.get_dummies(final_df['Borough'])

In [ ]:
final_df = final_df.join(one_hotBoroughs)

In [ ]:
final_df = final_df.loc[:,final_df.columns != 'Unnamed']

Encoding interests

In [ ]:
interests = final_df['Interest Categories']

zero_column = []
for i in range(len(interests)):
  zero_column.append(0)

one_hot_list = []
for i in range(10):
  one_hot_list.append(copy.deepcopy(zero_column))

categories = []
for item in short_cat_list:
  categories.append(item[1])


In [ ]:
for i in range(len(interests)):
  interests_list = interests[i].split(';')
  for item in interests_list:
    if item != '':
      one_hot_list[categories.index(item)][i] = 1

interests_dict = {'Health services': one_hot_list[0],
                  'Social services': one_hot_list[1],
                  'Advocacy and Justice': one_hot_list[2],
                  'Food and Agriculture': one_hot_list[3],
                  'Education': one_hot_list[4],
                  'Environment and Climate change': one_hot_list[5],
                  'Recreation and Leisure': one_hot_list[6],
                  'Technology and Innovation': one_hot_list[7],
                  'Outreach and Advertising': one_hot_list[8],
                  'Child services': one_hot_list[9]}

In [ ]:
interests_df = pd.DataFrame.from_dict(interests_dict) 

In [ ]:
interests_df['Organization/ City Agency/ Division Name'] = final_df['Organization/ City Agency/ Division Name']

In [ ]:
encoded_df = pd.merge(interests_df, final_df, on='Organization/ City Agency/ Division Name')

In [ ]:
encoded_df

Save encoded dataframe to CSV file

In [ ]:
#encoded_df.to_csv('/content/encoded_org_final_dataset.csv')